In [ ]:
!pip install pymupdf
!pip install tqdm
!pip install python-docx
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 13.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 4.9 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=6c89c274eda4c23f9403efb75d9f0a7ed3ae4da2f7313321beccf5094a59cdb2
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 12.6 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully unin

In [ ]:
import os
from pathlib import Path
import string
import nltk
nltk.download('stopwords')
import docx
import pytesseract
from PIL import Image
# https://stackoverflow.com/questions/22676/how-to-download-a-file-over-http
import urllib.request
# https://stackoverflow.com/questions/34837707/how-to-extract-text-from-a-pdf-file
# https://stackoverflow.com/questions/2693820/extract-images-from-pdf-without-resampling-in-python
import fitz  # pip install --upgrade pip; pip install --upgrade pymupdf
from tqdm import tqdm # pip install tqdm

from typing_extensions import ParamSpecArgs
import fitz # install using: pip install PyMuPDF
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import shutil
from collections import Counter, defaultdict

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# create new folder
newpathfoldersatu = 'foldersatu' 
if not os.path.exists(newpathfoldersatu) and not os.path.isfile(newpathfoldersatu):
    # create new folder
    os.makedirs(newpathfoldersatu)

newpathfolderdua = os.path.join(newpathfoldersatu,'folderdua')
if not os.path.exists(newpathfolderdua) and not os.path.isfile(newpathfolderdua):
    os.makedirs(newpathfolderdua)

newpathfoldertiga = os.path.join(newpathfoldersatu,'foldertiga')
if not os.path.exists(newpathfoldertiga) and not os.path.isfile(newpathfoldertiga):
    os.makedirs(newpathfoldertiga)

newpathfolderempat = os.path.join(newpathfoldertiga,'folderempat')
if not os.path.exists(newpathfolderempat) and not os.path.isfile(newpathfolderempat):
    os.makedirs(newpathfolderempat)

newpathfilesatu = os.path.join(newpathfoldersatu, 'filesatu.txt')
if not os.path.exists(newpathfilesatu) and not os.path.isdir(newpathfilesatu):
    # create new file
    with open(newpathfilesatu, "w+") as f: pass
    f = open(newpathfilesatu, "a")
    f.write("Text dalam file txt")
    f.write("\nText line kedua")
    f.close()

file_lima_url = "https://css4.pub/2015/textbook/somatosensory.pdf"
newpathfilelima = os.path.join(newpathfoldersatu, 'filelima.pdf')
if not os.path.exists(newpathfilelima) and not os.path.isdir(newpathfilelima):
    # download pdf file
    urllib.request.urlretrieve(file_lima_url, newpathfilelima)

# download folder "Contoh dokumen" berisi contoh dokumen dan dimasukkan ke foldersatu
!gdown --folder https://drive.google.com/drive/folders/1yEVRYLfkixpn5yunu_lzSPRFWLRC75oh?usp=sharing
contoh_dokumen_path = "Contoh dokumen"
contoh_dokumen_move_folder_to = os.path.join(newpathfoldersatu, contoh_dokumen_path)
shutil.move(contoh_dokumen_path, contoh_dokumen_move_folder_to) 

Retrieving folder list
Processing file 1nMKbBh3damMOWhfq78tX-ZhSP1sAxo8_ all-exchanges-strings.lc.txt
Processing file 1oG0JaZ76WhrYtOuI89C3ex8Fcxm6s4vx all-orgs-strings.lc.txt
Processing file 1XZqWDj2w8FgQkWKcRz9Y41N1JO6ORiYQ all-people-strings.lc.txt
Processing file 1BpkVzSwvu97AGX8Ga8gzdWIFWTU7a54w all-places-strings.lc.txt
Processing file 1Ze-AWHDAFWAg1pR9b6vzcaSzyDWnAQEd all-topics-strings.lc.txt
Processing file 1EVPPyozmoc9iu7FXMDbOrzf3UBsYi2-U cat-descriptions_120396.txt
Processing file 1BYXqlNJvTtXHZZRr6mD8z4nYFNuXM4EA feldman-cia-worldfactbook-data.txt
Processing file 1mzgRD-814A7i8cwW0Xara2qt1hAI57VC reut2-000.sgm
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1nMKbBh3damMOWhfq78tX-ZhSP1sAxo8_
To: /content/Contoh dokumen/all-exchanges-strings.lc.txt
100% 186/186 [00:00<00:00, 768kB/s]
Downloading...
From: https://drive.google.com/uc?id=1oG0JaZ76WhrYtOuI89C3ex8Fcxm6s4vx
To:

'foldersatu/Contoh dokumen'

In [ ]:
def crawl_and_scrape(initial_path, scrape_folder_save):
  # if initial path dont exist
  if not os.path.exists(initial_path) or not os.path.isdir(initial_path):
    raise ValueError("Initial path does not exist.")

  # if scrape folder does not exist
  if not os.path.exists(scrape_folder_save) or not os.path.isdir(scrape_folder_save):
    # create new folder
    os.makedirs(scrape_folder_save)

  # create new folder
  crawl_result_folder = os.path.join(scrape_folder_save, "crawl_result_folder")
  if not os.path.exists(crawl_result_folder) and not os.path.isfile(crawl_result_folder):
    # create new folder
    os.makedirs(crawl_result_folder)
  
  crawl_result_file = os.path.join(crawl_result_folder, 'crawl_result_file.txt')
  if not os.path.exists(crawl_result_file) and not os.path.isdir(crawl_result_file):
    # create new file
    with open(crawl_result_file, "w+") as f: pass

  current_layer_recursion_stack = [initial_path]
  files = []
  next_layer_recursion_stack = []
  folder_count_in_current_layer = 0

  idx_to_document = defaultdict(lambda: 0)
  index = 0
  idx = 1
  terms = {}
  term_frequency = defaultdict(lambda: 0)

  while len(current_layer_recursion_stack) > 0:
    index += 1
    files_this_folder = []
    current_folder = current_layer_recursion_stack.pop()
    folder_count_in_current_layer += 1
    folder_count_in_current_folder = 0
    file_count_in_current_folder = 0

    for name in os.listdir(current_folder):
      new_path = os.path.join(current_folder, name)
      if os.path.isfile(new_path):
        idx_to_document[idx] = name
        file_count_in_current_folder += 1
        files_this_folder.append(new_path)
        files.append(new_path)
        file_path = new_path
        if not os.path.exists(file_path) and not os.path.isdir(file_path):
          raise ValueError("File does not exist")
        file_extension = Path(file_path).suffix
        text = []
        if file_extension == ".png" or file_extension == ".jpg" or file_extension == ".jpeg":
          text_image = pytesseract.image_to_string(Image.open(file_path))
          text.append(text_image)
        if file_extension == ".txt":
          with open(file_path) as f:
            for line in f.readlines():
              text.append(line)
        elif file_extension == ".doc":
          # Belum implementasi
          pass
        elif file_extension == ".docx":
          doc = docx.Document(file_path)
          for para in doc.paragraphs:
            text.append(para.text)
        elif file_extension == ".pdf":
          with fitz.open(file_path) as doc:
              for page in doc:
                  text.append(page.get_text("text"))
        elif file_extension == ".sgm":
          with open(file_path) as f:
            for line in f.readlines():
              text.append(line)
        else:
          print(file_path)
          raise ValueError("Unsupported file format")

        # indexing
        documentid = index * idx
        temp = []
        ps = PorterStemmer()
        stop_words = stopwords.words('english')
        
        for i in range(len(text)):
          # https://stackoverflow.com/questions/8113782/split-string-on-whitespace-in-python
          text_to_process = text[i].split()

          # Text Transformation
          for j in range(len(text_to_process)):
            word_to_process = text_to_process[j]

            # remove punctuation
            word_to_process = word_to_process.translate(str.maketrans('', '', string.punctuation))

            # stopword removal
            if word_to_process in stop_words:
              continue
            else:
              word_to_process = ps.stem(word_to_process)
            if word_to_process != "":
              temp.append(word_to_process)

          if len(temp) != 1:
            terms[idx] = temp
        idx += 1
      else:
        folder_count_in_current_folder += 1
        next_layer_recursion_stack.append(new_path)
    

    if len(current_layer_recursion_stack) <= 0:
      current_layer_recursion_stack = next_layer_recursion_stack
      next_layer_recursion_stack = []
      folder_count_in_current_layer = 0
  return terms, idx_to_document


In [ ]:
import math

def unique(x):
  list_set = set(x)
  unique_list = (list(list_set))
  return unique_list

def get_idf(corpus):
  idf_dict=defaultdict(lambda: 0)
  terms_only = []
  for n in corpus:
    temp = corpus[n]
    for j in range(0, len(temp)):
      terms_only.append(temp[j])
  unique_word = unique(terms_only)

  for word in unique_word :
    count = 0
    for n in terms_only:
      if word == n:
        count += 1
    idf_dict[word] = (math.log((1+len(corpus))/(count+1)))+1
  return idf_dict

def Term_frequency(term, unique_word):
  term_frequency = defaultdict(lambda: 0)
  for word in unique_word :
    for i in range(len(term)):
      if word == term[i]:
        term_frequency[word] += 1
  return term_frequency

def tf_idf(tf, idf):
  tf_idf = (math.log(1+tf))*idf
  return tf_idf

def cosine(document_corpus, query_corpus, N, idf):
  tf_idf_document = defaultdict(lambda: 0)
  tf_idf_query = defaultdict(lambda: 0)
  score = {}
  query_unique_terms = unique(query_corpus)  
  query_tf = Term_frequency(query_corpus, query_unique_terms)
  for n in document_corpus:
    product = []
    document_terms = document_corpus[n]

    document_tf = Term_frequency(document_terms, query_unique_terms)

    for w in query_unique_terms:
      weight_document = tf_idf(document_tf[w], idf[w])
      weight_query = tf_idf(query_tf[w], idf[w])
      product.append(weight_query*weight_document)
    score[n] = sum(product)/len(document_terms)
  return score

In [ ]:
document_terms, index = crawl_and_scrape("foldersatu", "hasil_scraping")


In [ ]:
print(satu)

{1: ['text', 'dalam', 'file', 'txt', 'text', 'line', 'kedua'], 2: ['thi', 'sampl', 'document', 'showcas', 'pagebas', 'format', 'it', 'contain', 'chapter', 'wikibook', 'call', 'sensori', 'system', 'none', 'content', 'chang', 'articl', 'content', 'remov', 'anatomi', 'somatosensori', 'system', 'from', 'wikibooks1', 'our', 'somatosensori', 'system', 'consist', 'sensor', 'skin', 'sensor', 'muscl', 'tendon', 'joint', 'the', 'ceptor', 'skin', 'call', 'cutan', 'receptor', 'tell', 'us', 'temperatur', 'thermoreceptor', 'pressur', 'sur', 'face', 'textur', 'mechano', 'receptor', 'pain', 'nociceptor', 'the', 'receptor', 'muscl', 'joint', 'provid', 'inform', 'muscl', 'length', 'muscl', 'tension', 'joint', 'angl', 'cutan', 'receptor', 'sensori', 'inform', 'meissner', 'corpuscl', 'rapidli', 'adapt', 'affer', 'lead', 'adjust', 'grip', 'forc', 'object', 'lift', 'these', 'affer', 'respond', 'brief', 'burst', 'action', 'potenti', 'object', 'move', 'small', 'di', 'tanc', 'earli', 'stage', 'lift', 'in', 're

In [ ]:
query = ["A", "man", "is", "so", "small", "in", "the", "wilderness", "believe", "me", "The", "way", "how", 
                        "people", "is", "now", "we", "are", "not", "tailored", "to", "live", "there", "So", "when", "Stew", 
                        "say", "he", "stumble", "across", "a", "house", "in", "the", "middle", "of", "the", "mountain", 
                        "my", "ears", "prick", "up"]

In [ ]:
idf = get_idf(document_terms)
idf

defaultdict(<function __main__.get_idf.<locals>.<lambda>()>,
            {'lend': 0.17901944793016977,
             'bgfactbhutanimportcommoditiesfuel': 2.7047480922384253,
             'regan': 0.13979873477688853,
             'amoicpcaununctadunescounhcrunidounrwaupuwft': 2.7047480922384253,
             '420': 1.7884573603642702,
             'oldid5772': 2.7047480922384253,
             'chicheng': 2.2992829841302607,
             'bccanadalaunches12b': 2.7047480922384253,
             'unilev': 1.095310179804325,
             'herzegovinalandbounderiescroatiaserbia': 2.7047480922384253,
             'componentscomput': 2.7047480922384253,
             'diamondsgoldsilverzincmanganesetingermaniumuranium': 2.7047480922384253,
             'oldid5586': 2.7047480922384253,
             'newid84': 2.7047480922384253,
             '1846670': 2.7047480922384253,
             'bgfactparaguayimportcommoditiescapit': 2.7047480922384253,
             'newid875': 2.7047480922384253,
        

In [ ]:
rangking = cosine(document_terms, query, len(document_terms), idf)
rangking

{1: 0.0,
 2: 0.07504981891323427,
 3: 0.013592243991978442,
 4: 0.0,
 5: 0.0,
 6: 0.002835317589479854,
 7: 0.0018672764076150758,
 8: 0.0,
 9: 0.0,
 10: 0.0}